(TURN ON "RUN IN SEPARATE PROCESS")

# Top Projects Random Packages with Generated Descriptions

This notebook connects to the database, takes top 50 projects by stars, and for each project:
1. Retrieves a random package
2. Fetches the original package description
3. Generates a new description using OpenAI API
4. Prints both descriptions for comparison


In [1]:
@file:DependsOn("org.postgresql:postgresql:42.7.1")
@file:DependsOn("com.squareup.okhttp3:okhttp:4.12.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.15.2")


## Setup

First, let's import the necessary libraries and define our data classes.


In [2]:
import java.sql.Connection
import java.sql.DriverManager
import java.sql.ResultSet
import kotlin.random.Random
import java.io.File
import java.nio.file.Paths
import okhttp3.MediaType.Companion.toMediaType
import okhttp3.OkHttpClient
import okhttp3.Request
import okhttp3.RequestBody.Companion.toRequestBody
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue

/**
 * Data class representing a project.
 */
data class Project(
    val id: Int,
    val name: String,
    val stars: Int
)

/**
 * Data class representing a package.
 */
data class Package(
    val id: Int,
    val name: String,
    val groupId: String,
    val artifactId: String,
    val version: String,
    val description: String?
)

/**
 * Data classes for OpenAI API
 */
data class OpenAiRequest(
    val model: String,
    val messages: List<OpenAiMessage>,
)

// Update OpenAiMessage to include citations
data class OpenAiMessage(
    val role: String,
    val content: String,
    val refusal: String? = null,
    val annotations: List<OpenAiMessageAnnotation>? = null,
    val citations: List<Map<String, String>>? = null
)

// Add this new data class
data class OpenAiMessageAnnotation(
    val text: String? = null,
    val start_index: Int? = null,
    val end_index: Int? = null,
    val type: String? = null,
    val metadata: Map<String, Any>? = null
)

data class OpenAiResponse(
    val id: String?,
    val choices: List<OpenAiChoice>
)

data class OpenAiChoice(
    val message: OpenAiMessage,
    val index: Int,
    val logprobs: Any? = null,
    val finish_reason: String? = null
)

// OpenAI API constants
val OPENAI_API_URL = "https://api.openai.com/v1/chat/completions"
val OPENAI_API_KEY = System.getenv("OPENAI_API_KEY")
val OPENAI_MODEL = "gpt-4o-search-preview"



## Helper Functions

Now, let's define the helper functions to get the top projects by stars, a random package for each project, and to generate package descriptions using the OpenAI API.


In [3]:
/**
 * Get the top N projects by stars from the project_index materialized view.
 */
fun getTopProjectsByStars(connection: Connection, limit: Int): List<Project> {
    val projects = mutableListOf<Project>()

    val sql = """
        SELECT project_id, name, stars
        FROM project_index
        ORDER BY stars DESC
        LIMIT ?
    """

    connection.prepareStatement(sql).use { statement ->
        statement.setInt(1, limit)
        statement.executeQuery().use { resultSet ->
            while (resultSet.next()) {
                projects.add(
                    Project(
                        id = resultSet.getInt("project_id"),
                        name = resultSet.getString("name"),
                        stars = resultSet.getInt("stars")
                    )
                )
            }
        }
    }

    return projects
}

/**
 * Get a random package for a given project.
 */
fun getRandomPackageForProject(connection: Connection, projectId: Int): Package? {
    val sql = """
        SELECT id, name, group_id, artifact_id, version, description
        FROM package
        WHERE project_id = ?
    """

    val packages = mutableListOf<Package>()

    connection.prepareStatement(sql).use { statement ->
        statement.setInt(1, projectId)
        statement.executeQuery().use { resultSet ->
            while (resultSet.next()) {
                packages.add(
                    Package(
                        id = resultSet.getInt("id"),
                        name = resultSet.getString("name"),
                        groupId = resultSet.getString("group_id"),
                        artifactId = resultSet.getString("artifact_id"),
                        version = resultSet.getString("version"),
                        description = resultSet.getString("description")
                    )
                )
            }
        }
    }

    return if (packages.isNotEmpty()) {
        packages[Random.nextInt(packages.size)]
    } else {
        null
    }
}


## Database Connection

Now, let's set up the database connection parameters. In a real environment, these would be provided via environment variables or a configuration file.


In [4]:
@file:DependsOn("org.postgresql:postgresql:42.7.1")

// Database connection parameters
val jdbcUrl = System.getenv("DB_URL")
val username = System.getenv("DB_USERNAME")
val password = System.getenv("DB_PASSWORD")

// Load the PostgreSQL JDBC driver
Class.forName("org.postgresql.Driver")


class org.postgresql.Driver

## Load prompt from file

In [5]:
val promptPath = System.getenv("PROMPT_FILE_PATH")
val promptFile = File(promptPath)

In [6]:
fun sendWebSearchRequest(systemPrompt: String?, query: String, log: Boolean = false): String {
    val messages = mutableListOf<OpenAiMessage>()

    if (!systemPrompt.isNullOrBlank()) {
        messages.add(OpenAiMessage(role = "system", content = systemPrompt))
    }

    messages.add(
        OpenAiMessage(
            role = "user",
            content = query,
            // annotations = listOf(OpenAiMessageAnnotation("web_search"))
        )
    )

    val request = OpenAiRequest(
        model = OPENAI_MODEL,
        messages = messages,
    )

    val objectMapper = jacksonObjectMapper()
    objectMapper.configure(com.fasterxml.jackson.databind.DeserializationFeature.FAIL_ON_UNKNOWN_PROPERTIES, false)
    val requestJson = objectMapper.writeValueAsString(request)

    val client = OkHttpClient()

    // Create the request body
    val mediaType = "application/json; charset=utf-8".toMediaType()
    val requestBody = requestJson.toRequestBody(mediaType)
    if (log) {
        println("Request body: $requestJson\n\n\n")
    }
    // Create the request
    val httpRequest = Request.Builder()
        .url(OPENAI_API_URL)
        .addHeader("Authorization", "Bearer $OPENAI_API_KEY")
        .post(requestBody)
        .build()

    // Execute the request
    val response = client.newCall(httpRequest).execute()

    // Parse the response
    val responseBody = response.body?.string() ?: throw RuntimeException("Empty response body")

    if (log) {
        println("Response body: $responseBody\n\n\n")
    }
    val openAiResponse = objectMapper.readValue<OpenAiResponse>(responseBody)

    if (log) {
        openAiResponse.choices.first().message.citations?.let { citations ->
            println("\nSources used:")
            citations.forEach { citation ->
                println("- ${citation["url"]}")
            }
        }
    }

    // Return the generated description
    return openAiResponse.choices.first().message.content
}

fun cleanResponse(response: String): String {
    // Pattern to match anything in parentheses at the end of the string
    return response
        .trim()
        .replace("""\s*\(\[[^\]]*\]\([^)]*\)\)\s*$""".toRegex(), "").trim()
        .trim()
}


/**
 * Generate a description for a package using the OpenAI API.
 */
fun generatePackageDescription(packageEntity: Package): String {
    // Read the prompt from the file
    val promptTemplate = promptFile.readText()

    val userContent =
        """
        Package name: ${packageEntity.name}\n
        Group ID: ${packageEntity.groupId}\n
        Artifact ID: ${packageEntity.artifactId}\n
        Version: ${packageEntity.version}\n
        """

    val rawResult = sendWebSearchRequest(
        promptTemplate.replace("{packageName}", packageEntity.name),
        userContent.trimIndent(),
        log = true
    )

    return cleanResponse(rawResult)
}

In [7]:

// sendWebSearchRequest(null, "What is the current weather in Cape Town?")

## Main Execution

Finally, let's connect to the database, get the top projects, and print a random package for each project.


In [8]:
// Connect to the database
DriverManager.getConnection(jdbcUrl, username, password).use { connection ->
    // Get top 50 projects by stars
    val topProjects = getTopProjectsByStars(connection, 50)
    val topProjectsRandom = topProjects.shuffled().take(1)
    println("sdf")

    // For each project, get a random package, generate a description, and print both
    topProjectsRandom.forEach { project ->
        val randomPackage = getRandomPackageForProject(connection, project.id)
        if (randomPackage != null) {
            println("Project: ${project.name} (${project.stars} stars)")
            println("Random Package: ${randomPackage.groupId} ${randomPackage.artifactId}")
            println("Original Description: ${randomPackage.description ?: "No description available"}")

            // Generate a description using the OpenAI API
            try {
                val generatedDescription = generatePackageDescription(randomPackage)
                println("Generated Description: $generatedDescription")
            } catch (e: Exception) {
                println("Failed to generate description: ${e.message}")
            }

            println("---------------------------------------------------")
        } else {
            println("Project: ${project.name} (${project.stars} stars) - No packages found")
            println("---------------------------------------------------")
        }
    }
}


sdf
Project: kotlinx.serialization (5667 stars)
Random Package: org.jetbrains.kotlinx kotlinx-serialization-properties
Original Description: Kotlin multiplatform serialization runtime library
Request body: {"model":"gpt-4o-search-preview","messages":[{"role":"system","content":"You will be sent the description of a Kotlin package (Kotlin is a cross-platform programming language). Based on it, provide a brief 10-25 word summary of its purpose, main functionality and unique features. Follow rules:\n* Do not mention anything about Kotlin or Kotlin Multiplatform.\n* Do not mention the package's authors, creators or maintainers.\n* Try not to include information about compatibility with platforms.\n* Do not mention the license it uses (such as Apache 2.0, MIT and others).\n* Apply subject ellipsis, which involves removing explicit subjects when they're implied by context. Generate the summary without explicitly stating the package name or using placeholders like '...' for missing subjects.\